In [1]:
# pip install --quiet openai

In [2]:
# pip install --quiet  PyPDF2

In [3]:
# pip install --upgrade --quiet  langchain

In [4]:
# pip install --quiet langchain-openai faiss-cpu tiktoken

In [5]:
# pip install --quiet unstructured

In [6]:
# pip install --quiet  pdf2image

In [7]:
# pip install --quiet  pdfminer

In [8]:
import os
from openai import OpenAI

In [9]:
# import os
# from dotenv import load_dotenv
# load_dotenv()
# api_key= os.getenv("OPENAI_API_KEY")

In [10]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
import PyPDF2

In [12]:

# from langchain.document_loaders import TextLoader
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings
# from operator import itemgetter
# from langchain_community.vectorstores import FAISS
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough
# from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma, Pinecone
# from langchain.embeddings.openai import OpenAIEmbeddings

# from langchain.retrievers import ParentDocumentRetriever
# from langchain.storage import InMemoryStore
# from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.vectorstores import FAISS


In [13]:
# from langchain.retrievers import ParentDocumentRetriever
# from langchain.storage import InMemoryStore
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import Chroma
# from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
# import os
# import re
# from PyPDF2 import PdfReader
# from langchain.docstore.document import Document

# def load_pdf(file_path, start_page=10):
#     with open(file_path, 'rb') as file:
#         pdf_reader = PdfReader(file)
#         text = ''
#         for page_number, page in enumerate(pdf_reader.pages):
#             if page_number + 1 >= start_page:  # Start reading from the 10th page
#                 text += page.extract_text()
#         # Remove emails from the text
#         text_without_emails = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
#         # Ensure text is Unicode supported
#         unicode_text = text_without_emails.encode('utf-8', 'ignore').decode('utf-8')
#         return [Document(page_content=unicode_text, metadata={'file_name': os.path.basename(file_path)})]

# pdf_directory = "Must-Read-Books-for-Financial-Professionals"  # Replace this with the path to your directory containing PDF files
# pdf_files = [os.path.join(pdf_directory, file) for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

# docs = []
# for pdf_file in pdf_files:
#     docs.extend(load_pdf(pdf_file))



In [15]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=0)

In [16]:
# text = " "
# for i in docs:
#     text = text + "\n" + i.page_content

In [17]:

# chunks_splitted = text_splitter.create_documents([text])

In [18]:
import torch
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine

# Get our models - The package will take care of downloading the models automatically
# For best performance: Muennighoff/SGPT-5.8B-weightedmean-nli-bitfit
tokenizer = AutoTokenizer.from_pretrained("Muennighoff/SGPT-125M-weightedmean-nli-bitfit")
model = AutoModel.from_pretrained("Muennighoff/SGPT-125M-weightedmean-nli-bitfit")





c:\Users\DEEL\Desktop\Priyanshu\Ollama\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DEEL\Desktop\Priyanshu\Ollama\venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DEEL\.cache\huggingface\hub\models--Muennighoff--SGPT-125M-weightedmean-nli-bitfit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

In [19]:
def embeddings(text,model= model , tokenizer= tokenizer):
    batch_tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        last_hidden_state = model(**batch_tokens, output_hidden_states=True, return_dict=True).last_hidden_state
    weights = (
    torch.arange(start=1, end=last_hidden_state.shape[1] + 1)
    .unsqueeze(0)
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float().to(last_hidden_state.device))
    input_mask_expanded = (
    batch_tokens["attention_mask"]
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float())
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded * weights, dim=1)
    sum_mask = torch.sum(input_mask_expanded * weights, dim=1)
    
    embeddings = sum_embeddings / sum_mask
    return embeddings.detach().cpu().numpy()[0].tolist()
    

In [20]:
# trial = chunks_splitted[0:10]

In [21]:
# p=[]
# c=1
# for i in chunks_splitted:
#     x={"id": str(c), "values": embeddings(i.page_content), "metadata":{"text": i.page_content}}
#     p.append(x)
#     c=c+1 

In [22]:
# import dotenv
# from dotenv import load_dotenv
# load_dotenv()
# from pinecone import Pinecone
# pinecone_api_key = os.getenv("pinecone_api_key")
# pc = Pinecone(api_key = pinecone_api_key)
# index = pc.Index("finance")

In [23]:
# from load_pinecone_index import index 
# def upsert_data_in_pinecone(dict, index):
#     index.upsert(vectors=[dict], namespace="Fianl_set1")

# for i in range(len(p)):
#     upsert_data_in_pinecone(p[i], index= index)

In [24]:
import cohere
from dotenv import load_dotenv
import os
load_dotenv()
cohere_api_key= os.getenv("cohere_api_key")
co = cohere.Client(cohere_api_key)

In [25]:

from load_pinecone_index import index
def retriever_from_pinecone_then_reranking(query, index = index, embeddings= embeddings):
    embed_query = embeddings(query)
    pc = index.query(vector = embed_query,top_k=10, include_values = False, include_metadata = True, namespace="Fianl_set1")
    id_from_docs =[]
    text_retrieved =[]
    for i in pc["matches"]:
        id_from_docs.append(i["id"])
    for i in pc["matches"]:
        text_retrieved.append(i["metadata"]["text"])
    rerank_docs = co.rerank(query=query, documents=text_retrieved, top_n=5, model="rerank-english-v2.0")
    text =""
    text = text +"\n"+  rerank_docs[0].document["text"]
    text = text + "\n"+  rerank_docs[1].document["text"]
    text = text + "\n"+  rerank_docs[2].document["text"]
    text = text +"\n"+  rerank_docs[3].document["text"]
    text = text +"\n"+  rerank_docs[4].document["text"]
    response = co.summarize(
    text=text,
    model='command',
    length='long',
    extractiveness='high')
    return response.summary
    
    

In [26]:
retriever_from_pinecone_then_reranking("what is random walk")

"No technique, no matter how sophisticated, can consistently predict the future. That conclusion, if correct, represents to me (and I hope to you) a resounding confirmation of the random-walk approach. Recall that the weak form of the efficient-market hypothesis (the random-walk notion) says simply that the technical analysis of past price patterns to forecast the future is useless because any information from such an analysis will already have been incorporated in current market prices. If today's direction \x97 up or down, forward or backward \x97 does indeed predict tomorrow's step, then you will act on it today rather than tomorrow. Thus, if market participants were confident that the price of any security would double next week, the price would not reach that level over five working days."

In [27]:
def query_builder(retieved_Context, user_query):
    input_json = {"user_query": user_query, "context": retieved_Context}
    s=""" Attached below is the json format which consists of user_query and conext of the user_query which contain some retrieved information from literature regarding user_query, based on that context you will answer the user query. input json is {}""".format(input_json)
    return s

In [28]:
# query_builder(retriever_from_pinecone_then_reranking("what is random walk"), "what is random walk")

In [33]:
def chats():
    p=0
    chat_history = [
	{"user_name": "Chatbot", "text": "Hey! How can I help you today?"},]
    while True and p<5:
        input_query = input()
        input_query= query_builder(retriever_from_pinecone_then_reranking(input_query), input_query)
        response = co.chat(message=input_query, chat_history=chat_history)
        x={"user_name":"user", "text":input_query}
        chat_history.append(x)
        chat_history.append({"user_name": "Chatbot", "text":response.text})
        print(response.text)
        p=p+1

        
        

In [34]:
chats()

Okay, let's proceed using the provided context! Feel free to ask me any questions you may have regarding the meaning of random walks, and I will do my best to provide comprehensive answers.
Okay, the concept of a random walk is a theoretical model used to explain the behaviour of a process characterised by non-deterministic(random) movement influenced by external factors.

In the context of your provided text snippet, it seems that the random walk is being used as a metaphor to describe the behaviour of financial markets and the uncertainties faced by investors. The writer suggests that it is impossible to predict the direction of market trends with certainty, and any analysis focused on predicting future market movements based on historical data is futile. This idea is re-emphasized through the phrase "if today's direction (up or down, forward or backward) does indeed predict tomorrow's step, then you will act on it today rather than tomorrow." Here, the writer alludes to the concept 

In [2]:
from src.chat import main